In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np

In [2]:
dataset = pd.read_csv("dublinbikes_20200101_20200401.csv")

In [3]:
df = dataset
df

,STATION ID,TIME,LAST UPDATED,NAME,BIKE STANDS,AVAILABLE BIKE STANDS,AVAILABLE BIKES,STATUS,ADDRESS,LATITUDE,LONGITUDE
0,2,2020-01-01 06:25:02,2020-01-01 06:24:06,BLESSINGTON STREET,20,18,2,Open,Blessington Street,53.356770,-6.268140
1,2,2020-01-01 06:30:02,2020-01-01 06:24:06,BLESSINGTON STREET,20,18,2,Open,Blessington Street,53.356770,-6.268140
2,2,2020-01-01 06:35:02,2020-01-01 06:34:15,BLESSINGTON STREET,20,18,2,Open,Blessington Street,53.356770,-6.268140
3,2,2020-01-01 06:40:03,2020-01-01 06:37:22,BLESSINGTON STREET,20,20,0,Open,Blessington Street,53.356770,-6.268140
4,2,2020-01-01 06:45:02,2020-01-01 06:37:22,BLESSINGTON STREET,20,20,0,Open,Blessington Street,53.356770,-6.268140
...,...,...,...,...,...,...,...,...,...,...,...
2228273,117,2020-04-01 23:35:02,2020-04-01 23:32:46,HANOVER QUAY EAST,40,37,3,Open,Hanover Quay East,53.343655,-6.231755
2228274,117,2020-04-01 23:40:02,2020-04-01 23:32:46,HANOVER QUAY EAST,40,37,3,Open,Hanover Quay East,53.343655,-6.231755
2228275,117,2020-04-01 23:45:02,2020-04-01 23:42:54,HANOVER QUAY EAST,40,37,3,Open,Hanover Quay East,53.343655,-6.231755
2228276,117,2020-04-01 23:50:03,2020-04-01 23:42:54,HANOVER QUAY EAST,40,37,3,Open,Hanover Quay East,53.343655,-6.231755


In [22]:
%matplotlib qt 
import datetime
eval_df = df.copy()
eval_df['TIME'] = pd.to_datetime(eval_df['TIME'])
eval_df['date'] = eval_df['TIME'].dt.floor('D')

eval_df = eval_df.groupby(['STATION ID','date', 'STATUS', 'ADDRESS']).agg({'AVAILABLE BIKES': 'sum', 'AVAILABLE BIKE STANDS': 'sum'})
eval_df = eval_df.reset_index(level=[0,1,2,3])

top_address = eval_df.groupby('ADDRESS')['AVAILABLE BIKE STANDS'].sum().nlargest(20).index.tolist()
top_df = pd.DataFrame(columns=eval_df.columns)
for address in top_address:
    res = eval_df[eval_df['ADDRESS'] == address]
    top_df = top_df.append(res)

# normal plot show
top_df.set_index('date', inplace=True)
#top_df.groupby('ADDRESS')['AVAILABLE BIKES'].plot(Legend=True)

#fig = plt.figure(figsize=(10, 6))
#ax = fig.add_subplot(2,2)
fig, ax = plt.subplots(2, 2)
#ax = plt.figure(1).add_subplot(111)
# minimize the gap between time interval
def plot_graph(ax, top_address, label):
    for address in top_address:
         ax.plot(top_df[top_df['ADDRESS'] == address].index, top_df[top_df['ADDRESS'] == address]['AVAILABLE BIKE STANDS'], label = address)
    ax.xaxis.set_minor_locator(mdates.DayLocator(interval=1))
    ax.xaxis.set_major_locator(mdates.DayLocator(interval=3))
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    fig.autofmt_xdate()
    ax.grid(True)
    ax.set_title(f'top {label} dublin station against cycle count')
    ax.set_xlabel("Day")
    ax.set_ylabel("Total Cycle Usage")
    ax.legend(loc="upper left")
    ax.set_xlim([datetime.date(2020, 2, 1), datetime.date(2020, 3, 1)])
    #ax.set_xlim(['2020-02-01', '2020-03-01'])
    return ax
ax[0,0] = plot_graph(ax[0,0], top_address[0:5],'5')
ax[0,1] = plot_graph(ax[0,1], top_address[5:10], '5-10')
ax[1,0] = plot_graph(ax[1,0], top_address[10:15], '10-15')
ax[1,1] = plot_graph(ax[1,1], top_address[15:20], '15-20')
fig.tight_layout()

fig.subplots_adjust(hspace=0.2, wspace = 0.2)

plt.show()


In [21]:
import folium
from folium import plugins
from folium.plugins import HeatMap
import webbrowser

loc_df = df[['ADDRESS', 'LATITUDE', 'LONGITUDE']]
loc_df = loc_df.drop_duplicates()
#loc_df[loc_df['ADDRESS'] == 'Townsend Street', 'LATITUDE', 'LONGTITUDE']
city_latitude = float((loc_df.loc[loc_df['ADDRESS'].isin(['Townsend Street'])])['LATITUDE'])
city_longitude = float((loc_df.loc[loc_df['ADDRESS'].isin(['Townsend Street'])])['LONGITUDE'])

title = 'Dublin Bikes Stand Q1 - 2020'
title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(title)   

m = folium.Map([city_latitude, city_longitude], zoom_start=14)
for add in loc_df['ADDRESS']:
    stand_lat = float((loc_df.loc[loc_df['ADDRESS'].isin([add])])['LATITUDE'])
    stand_lon = float((loc_df.loc[loc_df['ADDRESS'].isin([add])])['LONGITUDE'])
    folium.Marker(location=[stand_lat, stand_lon], popup=f"{add}",  icon = folium.Icon(color='darkred', icon='bicycle', prefix='fa')).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.save("stand_location.html")
webbrowser.open("stand_location.html")

#folium.LayerControl().add_to(m)
#print(loc_df['ADDRESS'])

True

In [93]:
df1 = df
#df1.reset_index(inplace=True)
df1['formatted_time'] = df1['TIME'].dt.floor('h')
#df1['start_time2'] = df1['TIME'].apply(lambda x: x.replace(minute=0, second=0))
#df1
sum_df = df1.groupby(['STATION ID','formatted_time', 'STATUS', 'ADDRESS']).agg({'AVAILABLE BIKES': 'sum', 'AVAILABLE BIKE STANDS': 'sum'})
sum_df = sum_df.reset_index(level=[0,1])
sum_df['day_of_week'] = sum_df['formatted_time'].dt.strftime('%A')
sum_df['day_of_month'] = sum_df['formatted_time'].dt.strftime('%d')
sum_df['hour'] = sum_df['formatted_time'].dt.strftime('%H')
sum_df['month'] = sum_df['formatted_time'].dt.strftime('%m')
sum_df['week'] = sum_df['formatted_time'].dt.strftime('%w')
sum_df.to_csv("cleaned_dublinbikes_dataset.csv", encoding='utf-8')
#df1['month'] = df.cleareddate.dt.month
#df.groupby(['month','bts_name']).agg('sum').duration

In [163]:
df.tail

<bound method NDFrame.tail of          STATION ID                 TIME         LAST UPDATED  \
0                 2  2020-01-01 06:25:02  2020-01-01 06:24:06   
1                 2  2020-01-01 06:30:02  2020-01-01 06:24:06   
2                 2  2020-01-01 06:35:02  2020-01-01 06:34:15   
3                 2  2020-01-01 06:40:03  2020-01-01 06:37:22   
4                 2  2020-01-01 06:45:02  2020-01-01 06:37:22   
...             ...                  ...                  ...   
2228273         117  2020-04-01 23:35:02  2020-04-01 23:32:46   
2228274         117  2020-04-01 23:40:02  2020-04-01 23:32:46   
2228275         117  2020-04-01 23:45:02  2020-04-01 23:42:54   
2228276         117  2020-04-01 23:50:03  2020-04-01 23:42:54   
2228277         117  2020-04-01 23:55:02  2020-04-01 23:53:01   

                       NAME  BIKE STANDS  AVAILABLE BIKE STANDS  \
0        BLESSINGTON STREET           20                     18   
1        BLESSINGTON STREET           20               

In [4]:
df1 = df.copy()
df1['STATUS'] = df1['STATUS'].astype('category')
df1['LAST UPDATED'] = pd.to_datetime(df1['LAST UPDATED'])
df1['TIME'] = pd.to_datetime(df1['TIME'])
df1['formatted_time'] = df1['TIME'].dt.floor('h')
#ts = pd.to_datetime('2020/01/01')
#te = pd.to_datetime('2020/01/07')
#mask = (df1['TIME'] >= ts) & (df1['TIME'] <= te)
#df1.loc[mask]
#df['TIME'].max() - df['TIME'].min()

df1['day_of_week'] = df1['formatted_time'].dt.strftime('%A')
df1['day_of_month'] = df1['formatted_time'].dt.strftime('%d').astype(np.int64)
df1['hour'] = df1['formatted_time'].dt.strftime('%H').astype(np.int64)
df1['month'] = df1['formatted_time'].dt.strftime('%m').astype(np.int64)
df1['week'] = df1['formatted_time'].dt.strftime('%w').astype(np.int64)
numeric_columns = df1.select_dtypes(['int64']).columns
print(df1[numeric_columns].describe().T)
print(df1.select_dtypes(['category']).describe().T)
print(df1.dtypes)

                           count       mean        std   min   25%   50%  \
STATION ID             2228278.0  60.196628  33.598271   2.0  31.0  61.0   
BIKE STANDS            2228278.0  32.184342   7.666891  16.0  29.0  30.0   
AVAILABLE BIKE STANDS  2228278.0  20.456110  11.140868   0.0  12.0  20.0   
AVAILABLE BIKES        2228278.0  11.644086   9.980968   0.0   3.0  10.0   
day_of_month           2228278.0  15.902037   9.262035   1.0   8.0  16.0   
hour                   2228278.0  11.556068   6.876381   0.0   6.0  12.0   
month                  2228278.0   2.317890   0.729879   1.0   2.0   2.0   
week                   2228278.0   2.991054   1.976522   0.0   1.0   3.0   

                        75%    max  
STATION ID             90.0  117.0  
BIKE STANDS            40.0   40.0  
AVAILABLE BIKE STANDS  29.0   41.0  
AVAILABLE BIKES        18.0   40.0  
day_of_month           24.0   31.0  
hour                   17.0   23.0  
month                   3.0    4.0  
week               

In [7]:
df1

,STATION ID,TIME,LAST UPDATED,NAME,BIKE STANDS,AVAILABLE BIKE STANDS,AVAILABLE BIKES,STATUS,ADDRESS,LATITUDE,LONGITUDE,formatted_time,day_of_week,day_of_month,hour,month,week
0,2,2020-01-01 06:25:02,2020-01-01 06:24:06,BLESSINGTON STREET,20,18,2,Open,Blessington Street,53.356770,-6.268140,2020-01-01 06:00:00,Wednesday,1,6,1,3
1,2,2020-01-01 06:30:02,2020-01-01 06:24:06,BLESSINGTON STREET,20,18,2,Open,Blessington Street,53.356770,-6.268140,2020-01-01 06:00:00,Wednesday,1,6,1,3
2,2,2020-01-01 06:35:02,2020-01-01 06:34:15,BLESSINGTON STREET,20,18,2,Open,Blessington Street,53.356770,-6.268140,2020-01-01 06:00:00,Wednesday,1,6,1,3
3,2,2020-01-01 06:40:03,2020-01-01 06:37:22,BLESSINGTON STREET,20,20,0,Open,Blessington Street,53.356770,-6.268140,2020-01-01 06:00:00,Wednesday,1,6,1,3
4,2,2020-01-01 06:45:02,2020-01-01 06:37:22,BLESSINGTON STREET,20,20,0,Open,Blessington Street,53.356770,-6.268140,2020-01-01 06:00:00,Wednesday,1,6,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2228273,117,2020-04-01 23:35:02,2020-04-01 23:32:46,HANOVER QUAY EAST,40,37,3,Open,Hanover Quay East,53.343655,-6.231755,2020-04-01 23:00:00,Wednesday,1,23,4,3
2228274,117,2020-04-01 23:40:02,2020-04-01 23:32:46,HANOVER QUAY EAST,40,37,3,Open,Hanover Quay East,53.343655,-6.231755,2020-04-01 23:00:00,Wednesday,1,23,4,3
2228275,117,2020-04-01 23:45:02,2020-04-01 23:42:54,HANOVER QUAY EAST,40,37,3,Open,Hanover Quay East,53.343655,-6.231755,2020-04-01 23:00:00,Wednesday,1,23,4,3
2228276,117,2020-04-01 23:50:03,2020-04-01 23:42:54,HANOVER QUAY EAST,40,37,3,Open,Hanover Quay East,53.343655,-6.231755,2020-04-01 23:00:00,Wednesday,1,23,4,3


In [9]:
#df1 = df1.reset_index(level=[0])

%matplotlib qt
# Descriptive statistics for all the DateTime features
print(df1.select_dtypes(['datetime64[ns]']).describe(datetime_is_numeric=True).T)
# Descriptive statistics for all the categorial features
print(df1['AVAILABLE BIKES'].describe().T)
print(df1['AVAILABLE BIKE STANDS'].describe().T)
#df1['TIME'] = df1['TIME'].dt.floor('Min')
df1['STATUS'] = df1['STATUS'].astype('category')
print(df1['STATUS'].describe().T)

color = {'boxes': 'DarkGreen', 'whiskers': 'DarkOrange', 'medians': 'DarkBlue', 'caps': 'Gray'}

#df1['AVAILABLE BIKES'].plot(color= color, kind='box', figsize=(10, 10))
#plt.show()
df1['AVAILABLE BIKES'].plot(color= color, kind='box', figsize=(20, 20), title='Available bike usage', legend=True, grid=True)
plt.show()

                  count                          mean                 min  \
TIME            2228278 2020-02-25 08:41:56.862621184 2020-01-01 06:25:02   
LAST UPDATED    2228278 2020-02-25 08:36:12.593929216 2020-01-01 06:15:09   
formatted_time  2228278 2020-02-25 08:14:24.661230336 2020-01-01 06:00:00   

                               25%                 50%                 75%  \
TIME           2020-02-08 13:00:02 2020-02-26 06:55:02 2020-03-14 21:10:02   
LAST UPDATED   2020-02-08 12:55:48 2020-02-26 06:50:12 2020-03-14 21:06:10   
formatted_time 2020-02-08 13:00:00 2020-02-26 06:00:00 2020-03-14 21:00:00   

                               max  
TIME           2020-04-01 23:55:02  
LAST UPDATED   2020-04-01 23:54:04  
formatted_time 2020-04-01 23:00:00  
count    2.228278e+06
mean     1.164409e+01
std      9.980968e+00
min      0.000000e+00
25%      3.000000e+00
50%      1.000000e+01
75%      1.800000e+01
max      4.000000e+01
Name: AVAILABLE BIKES, dtype: float64
count    2.22827

In [13]:
hist_df = df1.copy()
hist_df = hist_df[['TIME', 'AVAILABLE BIKES', 'AVAILABLE BIKE STANDS', 'hour', 'day_of_month']]
numeric_columns = hist_df.select_dtypes(['int64']).columns
#numeric_columns = numeric_columns.drop('STATION ID')
# Histograms for all continuous features within the dataset

hist_df[numeric_columns].hist(figsize=(20, 20))
hist_df['day_of_month'].value_counts().sort_index().plot()


<AxesSubplot:title={'center':'day_of_month'}>

In [26]:
ts = pd.to_datetime('2020/01/01')
te = pd.to_datetime('2020/04/01')
mask = (df1['TIME'] >= ts) & (df1['TIME'] <= te)
pd.options.mode.chained_assignment = None
#df['TIME'].max() - df['TIME'].min()
suburb_point = "Merrion Square South"
suburb_df = df1.loc[mask]
suburb_df = suburb_df[suburb_df['ADDRESS'] == suburb_point]
suburb_dataset = suburb_df[['TIME', 'AVAILABLE BIKES']]
#fig, ax = plt.subplots()

numeric_columns = suburb_df.select_dtypes(['int64']).columns
print(suburb_df[numeric_columns].describe().T)

suburb_dataset['date'] = suburb_dataset['TIME'].dt.floor('T')
suburb_dataset = suburb_dataset.reset_index()
suburb_dataset.drop('TIME',axis=1, inplace=True)
time = suburb_dataset['date']
bikes = suburb_dataset['AVAILABLE BIKES']
#plot 2 week data for Merrion Square South region
fig = plt.figure()
ax = fig.add_subplot(111)
ax.xaxis.set_minor_locator(mdates.DayLocator(interval=1))
ax.xaxis.set_major_locator(mdates.DayLocator(interval=1))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
ax.scatter(time,bikes, color="red", marker=".", label='bike count')
ax.set_xlabel("date")
ax.set_ylabel("bike available")
fig.autofmt_xdate()
ax.grid(True)
ax.set_title(f'Bike usage for {suburb_point}')
ax.set_xlim([datetime.date(2020, 1, 1), datetime.date(2020, 4, 2)])
plt.show()
suburb_df

                         count        mean        std    min    25%    50%  \
STATION ID             20087.0  113.000000   0.000000  113.0  113.0  113.0   
BIKE STANDS            20087.0   40.000000   0.000000   40.0   40.0   40.0   
AVAILABLE BIKE STANDS  20087.0   31.271370  10.899708    0.0   29.0   36.0   
AVAILABLE BIKES        20087.0    8.704685  10.870020    0.0    1.0    4.0   
day_of_month           20087.0   16.113456   9.153428    1.0    8.0   16.0   
hour                   20087.0   11.555583   6.875501    0.0    6.0   12.0   
month                  20087.0    2.289491   0.707410    1.0    2.0    2.0   
week                   20087.0    2.990541   1.990246    0.0    1.0    3.0   

                         75%    max  
STATION ID             113.0  113.0  
BIKE STANDS             40.0   40.0  
AVAILABLE BIKE STANDS   39.0   40.0  
AVAILABLE BIKES         11.0   40.0  
day_of_month            24.0   31.0  
hour                    17.0   23.0  
month                    3.0   

,STATION ID,TIME,LAST UPDATED,NAME,BIKE STANDS,AVAILABLE BIKE STANDS,AVAILABLE BIKES,STATUS,ADDRESS,LATITUDE,LONGITUDE,formatted_time,day_of_week,day_of_month,hour,month,week
17621,113,2020-01-01 06:25:02,2020-01-01 06:22:01,MERRION SQUARE SOUTH,40,36,4,Open,Merrion Square South,53.338615,-6.248606,2020-01-01 06:00:00,Wednesday,1,6,1,3
17622,113,2020-01-01 06:30:02,2020-01-01 06:22:01,MERRION SQUARE SOUTH,40,36,4,Open,Merrion Square South,53.338615,-6.248606,2020-01-01 06:00:00,Wednesday,1,6,1,3
17623,113,2020-01-01 06:35:02,2020-01-01 06:32:10,MERRION SQUARE SOUTH,40,36,4,Open,Merrion Square South,53.338615,-6.248606,2020-01-01 06:00:00,Wednesday,1,6,1,3
17624,113,2020-01-01 06:40:03,2020-01-01 06:32:10,MERRION SQUARE SOUTH,40,36,4,Open,Merrion Square South,53.338615,-6.248606,2020-01-01 06:00:00,Wednesday,1,6,1,3
17625,113,2020-01-01 06:45:02,2020-01-01 06:42:19,MERRION SQUARE SOUTH,40,36,4,Open,Merrion Square South,53.338615,-6.248606,2020-01-01 06:00:00,Wednesday,1,6,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2195441,113,2020-03-31 23:35:02,2020-03-31 23:32:57,MERRION SQUARE SOUTH,40,30,10,Open,Merrion Square South,53.338615,-6.248606,2020-03-31 23:00:00,Tuesday,31,23,3,2
2195442,113,2020-03-31 23:40:02,2020-03-31 23:32:57,MERRION SQUARE SOUTH,40,30,10,Open,Merrion Square South,53.338615,-6.248606,2020-03-31 23:00:00,Tuesday,31,23,3,2
2195443,113,2020-03-31 23:45:02,2020-03-31 23:43:07,MERRION SQUARE SOUTH,40,30,10,Open,Merrion Square South,53.338615,-6.248606,2020-03-31 23:00:00,Tuesday,31,23,3,2
2195444,113,2020-03-31 23:50:03,2020-03-31 23:43:07,MERRION SQUARE SOUTH,40,30,10,Open,Merrion Square South,53.338615,-6.248606,2020-03-31 23:00:00,Tuesday,31,23,3,2


In [19]:
# trial model don't refer

from sklearn.linear_model import Ridge 
from sklearn.model_selection import train_test_split 
import math


def test_preds(t, y, q, dd, lag, dt):
    #q−step ahead prediction
    stride=1 
    XX=y[0:y.size-q-lag*dd:stride]

    for i in range(1,lag): 
        X=y[i*dd:y.size-q-(lag-i)*dd:stride] 
        XX=np.column_stack((XX,X)) 
        yy=y[lag*dd+q::stride]
        tt=t[lag*dd+q::stride]
        
        yy.index = range(len(yy.index))
        print(yy.index)
        start = yy.index.start
        stop = yy.index.stop
        #Xtrain, Xtest, Ytrain, Ytest = train_test_split(XX, yy,test_size=0.2)
        train, test = train_test_split(np.arange(start,stop),test_size=0.2)
        
        #model = Ridge(fit_intercept=False).fit(Xtrain, Ytrain)
        model = Ridge(fit_intercept=False).fit(XX[train], yy[train])
        print(model.intercept_, model.coef_) 
        #print(model.score(Xtest, Ytest))
        print(model.score(XX[test], yy[test]))
        #plotting
        y_pred = model.predict(XX) 
        plt.scatter(t, y, color="black")
        plt.scatter(tt, y_pred, color="lightblue") 
        plt.xlabel("time (days)")
        plt.ylabel("#bikes") 
        plt.legend(["training data","predictions"],loc="upper right") 
        day=math.floor(24*60*60/dt) # number of samples per day 
        # print(day)
        # print((lag*dd+q)/day,(lag*dd+q)/day+2)
        #plt.xlim(((lag*dd+q)/day,(lag*dd+q)/day+2))
        # print("---------", plt.axis())
        plt.show()

t_full = pd.array(pd.DatetimeIndex(suburb_df.iloc[:,1]).astype(np.int64))/1000000000
dt = t_full[1]-t_full[0]
# test_preds(time, bikes, q=10,dd=1,lag=3, dt=dt)
# d=math.floor(24*60*60/dt) # number of samples per day 
# test_preds(time, bikes, q=d,dd=d,lag=3, dt=dt)
w=math.floor(7*24*60*60/dt) # number of samples per day 
test_preds(time, bikes, q=w,dd=w,lag=3,dt= dt)

RangeIndex(start=0, stop=9063, step=1)
0.0 [ 0.68221513 -0.06277519]
0.2011209083257467
RangeIndex(start=0, stop=9063, step=1)
0.0 [ 0.65940031 -0.07630622  0.04856598]
0.15343679944291633


C:\Users\rkhna\AppData\Local\Temp/ipykernel_34468/3797443078.py:45: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  t_full = pd.array(pd.DatetimeIndex(suburb_df.iloc[:,1]).astype(np.int64))/1000000000


In [258]:
from sklearn.metrics import r2_score

q=10
lag=3
stride=1
w=math.floor(7*24*60*60/dt)
length = bikes.size - w - lag * w - q
print(bikes.size, w, lag, q, length)
XX=bikes[q:q+length:stride]
#week
for i in range(1,lag):
    X=bikes[i*w+q:i*w+q+length:stride]
    XX=np.column_stack((XX,X))
d=math.floor(24*60*60/dt)
#days
for i in range(0,lag):
    X=bikes[i*d+q:i*d+q+length:stride]
    XX=np.column_stack((XX,X))

for i in range(0,lag):
    X=bikes[i:i+length:stride]
    XX=np.column_stack((XX,X))

yy=bikes[lag*w+w+q:lag*w+w+q+length:stride] 
tt=time[lag*w+w+q:lag*w+w+q+length:stride]

yy.reset_index(drop=True, inplace=True)
tt.reset_index(drop=True, inplace=True)

train, test = train_test_split(np.arange(0,yy.size),test_size=0.2)
model = Ridge(fit_intercept=False).fit(XX[train], yy[train])
print(model.intercept_, model.coef_)
y_pred = model.predict(XX)
print(model.score(XX[test], yy[test]))

from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(yy,y_pred))
print(rmse)
print("------")
print(yy.count())
print(y_pred.size)
print("-------")
print(r2_score(yy,y_pred))
plt.scatter(time, bikes, color="black")
plt.scatter(tt, y_pred, color="blue")
plt.xlabel("time (days)")
plt.ylabel("#bikes")
plt.legend(["training data","predictions"],loc="upper right")
day=math.floor(24*60*60/dt)
#plt.xlim((4*7,4*7+4))
plt.show()

17127 2016 3 10 9053
0.0 [ 0.27633477 -0.10394769  0.03254828  0.27633477 -0.06160802  0.11781584
  0.46090312 -0.0352891  -0.30496061]
0.2656771399752311
7.808036094970542
------
9053
9053
-------
0.22692995823821482


In [18]:
from sklearn.metrics import r2_score

q=10
lag=3
stride=1
w=math.floor(7*24*60*60/dt)
length = bikes.size - w - lag * w - q
print(bikes.size, w, lag, q, length)
XX=bikes[q:q+length:stride]
#week
for i in range(1,lag):
    X=bikes[i*w+q:i*w+q+length:stride]
    XX=np.column_stack((XX,X))
d=math.floor(24*60*60/dt)
#days
for i in range(0,lag):
    X=bikes[i*d+q:i*d+q+length:stride]
    XX=np.column_stack((XX,X))

for i in range(0,lag):
    X=bikes[i:i+length:stride]
    XX=np.column_stack((XX,X))

yy=bikes[lag*w+w+q:lag*w+w+q+length:stride] 
tt=time[lag*w+w+q:lag*w+w+q+length:stride]

yy.reset_index(drop=True, inplace=True)
tt.reset_index(drop=True, inplace=True)

train, test = train_test_split(np.arange(0,yy.size),test_size=0.2)

coeff = []
alphas = []
errors = []

C = [0.0001, 0.00015, 0.0002, 0.0003, 0.0004, 0.0005, 0.001, 0.005, 0.05, 0.5, 1, 5,10,50,100,1000 ] #,1500 ,2000,3000,4000,10000] [0.0001, 0.0005, 
for Ci in C:
    a = 1/(2*Ci)
    print(Ci, a)
    model = Ridge(alpha= a,max_iter=7000).fit(XX[train], yy[train])
    print(model.intercept_, model.coef_)
    coeff.append(model.coef_)
    alphas.append(a)
    y_pred = model.predict(XX)
    score = model.score(XX[test], yy[test])
    #r2_score = r2_score(yy,y_pred)
    from sklearn.model_selection import cross_val_score
    # scores = cross_val_score(model, XX[test], yy[test], cv=5, scoring='f1')
    mse = mean_squared_error(yy,y_pred)
    errors.append(mse)
    print("SCORE: {0:.3f}, MSE:{1:.2f}, RMSE:{2:.2f}"
   .format(score,  mse,np.sqrt(mse)))
    #print(r2_score(yy,y_pred))
    print(f'r2_score - {r2_score(yy,y_pred)}')

plt.close("all")
fig, (ax1, ax2) = plt.subplots(1, 2)
#fig=plt.figure()
#ax1=fig.add_subplot(111)
ax1.plot(alphas,coeff)
ax1.set_xlabel('Alpha (Regularization Parameter)')
ax1.set_ylabel('Beta (Predictor Coefficients)')
ax1.set_title('Ridge Coefficients vs Regularization Parameters')
ax1.axis('tight')
fig.show()

#fig=plt.figure()
#ax2=fig.add_subplot(111)

ax2 = plt.gca()
ax2.ticklabel_format(useOffset=False)
errors = [round(num, 5) for num in errors]
ax2.plot(alphas, errors)
ax2.set_xlabel("alpha")
ax2.set_ylabel("error")
ax2.set_title("Coefficient error as a function of the regularization")
ax2.axis("tight")
fig.show()
# ax2.set_yticklabels(errors)
print(errors)
print(alphas)

17127 2016 3 10 9053
0.0001 5000.0
4.3109920210773796 [ 0.25460743 -0.11122464 -0.04237303  0.25460743 -0.04711408  0.012809
  0.06807063  0.00942116  0.03495859]
SCORE: 0.324, MSE:50.80, RMSE:7.13
r2_score - 0.35578384863880086
0.00015 3333.3333333333335
4.311089635705649 [ 0.25875762 -0.11296974 -0.04309229  0.25875762 -0.04795185  0.01315152
  0.08220834 -0.00353207  0.02834826]
SCORE: 0.324, MSE:50.80, RMSE:7.13
r2_score - 0.3558087044274768
0.0002 2500.0
4.31094545279233 [ 0.26112419 -0.11386263 -0.04346287  0.26112419 -0.04837819  0.01332973
  0.09397953 -0.01450793  0.02431036]
SCORE: 0.324, MSE:50.80, RMSE:7.13
r2_score - 0.3558179888230295
0.0003 1666.6666666666667
4.310555888099172 [ 0.26381353 -0.11477194 -0.04384342  0.26381353 -0.04880872  0.01351451
  0.11228587 -0.03261482  0.02027469]
SCORE: 0.324, MSE:50.80, RMSE:7.13
r2_score - 0.3558237746923023
0.0004 1250.0
4.310212381828225 [ 0.26533547 -0.11523445 -0.0440386   0.26533547 -0.04902506  0.01361035
  0.12591894 -0.04

In [14]:
from sklearn.metrics import r2_score
from sklearn.linear_model import Lasso
import math
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
t_full = pd.array(pd.DatetimeIndex(suburb_df.iloc[:,1]).astype(np.int64))/1000000000
dt = t_full[1]-t_full[0]

%matplotlib qt 

q=2
lag=3
stride=1
w=math.floor(7*24*60*60/dt)
length = bikes.size - w - lag * w - q
print(bikes.size, w, lag, q, length)
XX=bikes[q:q+length:stride]
#week
for i in range(1,lag):
    X=bikes[i*w+q:i*w+q+length:stride]
    XX=np.column_stack((XX,X))
d=math.floor(24*60*60/dt)
#days
for i in range(0,lag):
    X=bikes[i*d+q:i*d+q+length:stride]
    XX=np.column_stack((XX,X))

for i in range(0,lag):
    X=bikes[i:i+length:stride]
    XX=np.column_stack((XX,X))

yy=bikes[lag*w+w+q:lag*w+w+q+length:stride] 
tt=time[lag*w+w+q:lag*w+w+q+length:stride]

yy.reset_index(drop=True, inplace=True)
tt.reset_index(drop=True, inplace=True)

train, test = train_test_split(np.arange(0,yy.size),test_size=0.2)

coeff = []
alphas = []
errors = []
scores = []

C = [1, 3, 5, 10, 50, 75, 100, 500, 1000, 2000, 10000]

for Ci in C:
    a = 1/(2*Ci)
    print(Ci, a)
    model = Lasso(alpha= a,max_iter=7000).fit(XX[train], yy[train])
    print(model.intercept_, model.coef_)
    coeff.append(model.coef_)
    alphas.append(a)
    y_pred = model.predict(XX)
    score = model.score(XX[test], yy[test])
    #r2_score = r2_score(yy,y_pred)
    from sklearn.model_selection import cross_val_score
    # scores = cross_val_score(model, XX[test], yy[test], cv=5, scoring='f1')
    mse = mean_squared_error(yy,y_pred)
    errors.append(np.sqrt(mse))
    print("SCORE: {0:.3f}, MSE:{1:.2f}, RMSE:{2:.2f}"
   .format(score,  mse,np.sqrt(mse)))
    #print(r2_score(yy,y_pred))
    scores.append(r2_score(yy,y_pred))
    print(f'r2_score - {r2_score(yy,y_pred)}')

    
plt.close("all")
fig, (ax1, ax2, ax3) = plt.subplots(1, 3)
#fig=plt.figure()
#ax1=fig.add_subplot(111)
ax1.plot(alphas,coeff)
ax1.set_xlabel('Alpha (Regularization Parameter)')
ax1.set_ylabel('Beta (Predictor Coefficients)')
ax1.set_title('Ridge Coefficients vs Regularization Parameters')
ax1.axis('tight')


# ax2 = plt.gca()
# ax2.ticklabel_format(useOffset=False)
errors = [round(num, 5) for num in errors]
ax2.plot(alphas, errors)
ax2.set_xlabel("alpha")
ax2.set_ylabel("RMSE error")
ax2.set_title("Coefficient error as a function of the regularization")
ax2.axis("tight")
                  

scores = [round(num, 5) for num in scores]
ax3.plot(alphas, scores)
ax3.set_xlabel("alpha")
ax3.set_ylabel("r2 score")
ax3.set_title("r2 score vs regularization Parameter")
ax3.axis("tight")
fig.show()

C:\Users\rkhna\AppData\Local\Temp/ipykernel_33108/858898009.py:7: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  t_full = pd.array(pd.DatetimeIndex(suburb_df.iloc[:,1]).astype(np.int64))/1000000000


17127 2016 3 2 9061
1 0.5
4.336606491178786 [ 0.25470318 -0.09146012 -0.04989146  0.11855469 -0.02862438  0.
  0.2164855   0.          0.        ]
SCORE: 0.339, MSE:50.88, RMSE:7.13
r2_score - 0.35473983384264396
3 0.16666666666666666
4.351537232987342 [ 0.25311701 -0.10594188 -0.0606678   0.1395764  -0.03866111  0.00353186
  0.22638685  0.          0.        ]
SCORE: 0.338, MSE:50.85, RMSE:7.13
r2_score - 0.35503607427074235
5 0.1
4.350484396567776 [ 0.25203472 -0.1089636  -0.06272878  0.14381601 -0.04180004  0.0055452
  0.22940323  0.          0.        ]
SCORE: 0.338, MSE:50.85, RMSE:7.13
r2_score - 0.35505183644238736
10 0.05
4.349699374039105 [ 0.25111542 -0.11123047 -0.06427518  0.14699674 -0.04415462  0.00705546
  0.2317729   0.          0.        ]
SCORE: 0.338, MSE:50.85, RMSE:7.13
r2_score - 0.3550453756769206
50 0.01
4.349081774036716 [ 2.50055546e-01 -1.13045717e-01 -6.55143011e-02  1.49544426e-01
 -4.60394422e-02  8.26443447e-03  2.33992032e-01  0.00000000e+00
  8.46803087

C:\Users\rkhna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 80435.26611908094, tolerance: 56.58859116997804
  model = cd_fast.enet_coordinate_descent(


4.348722116666348 [ 0.34732192 -0.11344964 -0.06577593  0.15010031 -0.0464591   0.00853136
  0.24584976 -0.02357839 -0.08525806]
SCORE: 0.337, MSE:50.85, RMSE:7.13
r2_score - 0.35502447880898125
1000 0.0005


C:\Users\rkhna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 80422.20195670113, tolerance: 56.58859116997804
  model = cd_fast.enet_coordinate_descent(


4.348748233150126 [ 0.40038258 -0.11347133 -0.06578654  0.15012746 -0.04648154  0.00854507
  0.24947203 -0.03089083 -0.13461831]
SCORE: 0.337, MSE:50.85, RMSE:7.13
r2_score - 0.35502395028058187
2000 0.00025


C:\Users\rkhna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 80401.13495218538, tolerance: 56.58859116997804
  model = cd_fast.enet_coordinate_descent(


4.348761291392007 [ 0.42691291 -0.11348217 -0.06579185  0.15014104 -0.04649276  0.00855192
  0.25128317 -0.03454706 -0.15929844]
SCORE: 0.337, MSE:50.85, RMSE:7.13
r2_score - 0.3550236174867475
10000 5e-05
4.34877173798553 [ 0.44813718 -0.11349084 -0.06579609  0.1501519  -0.04650174  0.0085574
  0.25273208 -0.03747204 -0.17904254]
SCORE: 0.337, MSE:50.85, RMSE:7.13
r2_score - 0.3550233183574554


C:\Users\rkhna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 80247.12546926137, tolerance: 56.58859116997804
  model = cd_fast.enet_coordinate_descent(


In [30]:
# try this 

from sklearn.metrics import r2_score
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
import math

t_full = pd.array(pd.DatetimeIndex(suburb_df.iloc[:,1]).astype(np.int64))/1000000000
dt = t_full[1]-t_full[0]

q=10
lag=3
stride=1
w=math.floor(7*24*60*60/dt)
length = bikes.size - w - lag * w - q
print(bikes.size, w, lag, q, length)
XX=bikes[q:q+length:stride]
#week
for i in range(1,lag):
    X=bikes[i*w+q:i*w+q+length:stride]
    XX=np.column_stack((XX,X))
d=math.floor(24*60*60/dt)
#days
for i in range(0,lag):
    X=bikes[i*d+q:i*d+q+length:stride]
    XX=np.column_stack((XX,X))

for i in range(0,lag):
    X=bikes[i:i+length:stride]
    XX=np.column_stack((XX,X))

yy=bikes[lag*w+w+q:lag*w+w+q+length:stride] 
tt=time[lag*w+w+q:lag*w+w+q+length:stride]

yy.reset_index(drop=True, inplace=True)
tt.reset_index(drop=True, inplace=True)


train, test = train_test_split(np.arange(0,yy.size),test_size=0.2)

coeff = []
alphas = []
errors = []
c_mean_error = []
c_std_error = []

C = [0.01, 0.1, 1, 3, 5, 10, 50,  100]

for Ci in C:
    c_temp = []
    mean_error = []
    std_error = []
    XPoly = PolynomialFeatures(5).fit_transform(XX)
    a = 1/(2*Ci)
    print(Ci, a)
    model = Lasso(alpha= a).fit(XPoly[train], yy[train])
    print("coefficient " ,model.intercept_, model.coef_)
    coeff.append(model.coef_)
    alphas.append(a)
    y_pred = model.predict(XPoly)
    score = model.score(XPoly[test], yy[test])
    # scores = cross_val_score(model, XX[test], yy[test], cv=5, scoring='f1')
    mse = mean_squared_error(yy,y_pred)
    errors.append(np.sqrt(mse))
    print("SCORE: {0:.3f}, MSE:{1:.2f}, RMSE:{2:.2f}"
   .format(score,  mse,np.sqrt(mse)))
    print(f'r2_score - {r2_score(yy,y_pred)}')

plt.close("all")
fig, (ax1, ax2) = plt.subplots(1, 2)
#fig=plt.figure()
#ax1=fig.add_subplot(111)
ax1.plot(alphas,coeff)
ax1.set_xlabel('Alpha (Regularization Parameter)')
ax1.set_ylabel('Beta (Predictor Coefficients)')
ax1.set_title('Ridge Coefficients vs Regularization Parameters')
ax1.axis('tight')

ax2 = plt.gca()
ax2.ticklabel_format(useOffset=False)
errors = [round(num, 5) for num in errors]
ax2.plot(alphas, errors)
ax2.set_xlabel("alpha")
ax2.set_ylabel("RMSE error")
ax2.set_title("Coefficient error as a function of the regularization")
ax2.axis("tight")
fig.show()


C:\Users\rkhna\AppData\Local\Temp/ipykernel_33108/1230507562.py:12: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  t_full = pd.array(pd.DatetimeIndex(suburb_df.iloc[:,1]).astype(np.int64))/1000000000


17127 2016 3 10 9053
0.01 50.0


C:\Users\rkhna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 87756.07314860026, tolerance: 56.63180669704499
  model = cd_fast.enet_coordinate_descent(


coefficient  5.330678600424896 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ... -2.43765210e-09
  5.11695174e-09  1.32131284e-08]
SCORE: 0.691, MSE:23.71, RMSE:4.87
r2_score - 0.6993748482808928
0.1 5.0


C:\Users\rkhna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 83957.260244914, tolerance: 56.63180669704499
  model = cd_fast.enet_coordinate_descent(


...

coefficient  4.762197818531412 [ 0.00000000e+00  0.00000000e+00  3.22731218e-01 ... -1.40214007e-09
  6.14679642e-09  1.38104458e-08]
SCORE: 0.719, MSE:21.71, RMSE:4.66
r2_score - 0.7246668868815593
50 0.01


C:\Users\rkhna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 77683.45569774426, tolerance: 56.63180669704499
  model = cd_fast.enet_coordinate_descent(


coefficient  4.724903510307854 [ 0.00000000e+00  3.09942239e-01  3.37478691e-01 ... -1.60215744e-09
  5.97261257e-09  1.36590519e-08]
SCORE: 0.719, MSE:21.70, RMSE:4.66
r2_score - 0.7248198318842283
100 0.005


C:\Users\rkhna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 77604.49859229225, tolerance: 56.63180669704499
  model = cd_fast.enet_coordinate_descent(


coefficient  4.727801536094587 [ 0.00000000e+00  3.83509187e-01  3.36546073e-01 ... -1.64690227e-09
  5.93744111e-09  1.36309178e-08]
SCORE: 0.719, MSE:21.70, RMSE:4.66
r2_score - 0.7248391041425568


In [24]:
# GET LAG VALUE (3)

from sklearn.metrics import r2_score

q=2
lag_range = [1,2,3,4]
stride=1
Ci = 10
scores = []
errors = []
for lag in lag_range:
    w=math.floor(7*24*60*60/dt)
    length = bikes.size - w - lag * w - q
    print("intial set : ", bikes.size, w, lag, q, length)
    XX=bikes[q:q+length:stride]
    #week
    for i in range(1,lag):
        X=bikes[i*w+q:i*w+q+length:stride]
        XX=np.column_stack((XX,X))
    d=math.floor(24*60*60/dt)
    #days
    for i in range(0,lag):
        X=bikes[i*d+q:i*d+q+length:stride]
        XX=np.column_stack((XX,X))

    for i in range(0,lag):
        X=bikes[i:i+length:stride]
        XX=np.column_stack((XX,X))

    yy=bikes[lag*w+w+q:lag*w+w+q+length:stride] 
    tt=time[lag*w+w+q:lag*w+w+q+length:stride]

    yy.reset_index(drop=True, inplace=True)
    tt.reset_index(drop=True, inplace=True)

    train, test = train_test_split(np.arange(0,yy.size),test_size=0.2)

    a = 1/(2*Ci)
    print("lag value - ", lag)
    model = Lasso(alpha= a).fit(XX[train], yy[train])
    y_pred = model.predict(XX)
    score = model.score(XX[test], yy[test])
    from sklearn.model_selection import cross_val_score
    mse = mean_squared_error(yy,y_pred)
    errors.append(mse)
    r2 = r2_score(yy,y_pred)
    scores.append(r2)
    print("SCORE: {0:.3f}, MSE:{1:.2f}, RMSE:{2:.2f}"
    .format(score,  mse,np.sqrt(mse)))
    print(f'r2_score - {r2}')

plt.close("all")
fig, (ax1, ax2) = plt.subplots(1, 2)
#fig=plt.figure()
#ax1=fig.add_subplot(111)
ax1.plot(lag_range,scores)
ax1.set_xlabel('Alpha (lag range)')
ax1.set_ylabel('Beta (Score)')
ax1.set_title('Model Score vs Lag Value')
ax1.axis('tight')

# fig=plt.figure()
# ax2=fig.add_subplot(111)

ax2 = plt.gca()
ax2.ticklabel_format(useOffset=False)
errors = [round(num, 5) for num in errors]

ax2.plot(lag_range, errors)
ax2.set_xlabel("lag")
ax2.set_ylabel("error")
ax2.set_title("Error vs Lag")
ax2.axis("tight")
fig.show()
# ax2.set_yticklabels(errors)
print(scores)
print(errors)
print(lag_range)

intial set :  17127 2016 1 2 13093
lag value -  1
SCORE: 0.386, MSE:54.43, RMSE:7.38
r2_score - 0.37570966353312474
intial set :  17127 2016 2 2 11077
lag value -  2
SCORE: 0.368, MSE:54.77, RMSE:7.40
r2_score - 0.35083517871239966
intial set :  17127 2016 3 2 9061
lag value -  3
SCORE: 0.374, MSE:50.87, RMSE:7.13
r2_score - 0.35483761979089035
intial set :  17127 2016 4 2 7045
lag value -  4
SCORE: 0.171, MSE:45.47, RMSE:6.74
r2_score - 0.20744485740845964
[0.37570966353312474, 0.35083517871239966, 0.35483761979089035, 0.20744485740845964]
[54.42892, 54.7744, 50.86836, 45.47166]
[1, 2, 3, 4]


In [29]:
from sklearn.metrics import r2_score
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
import math

q=2
lag=3
stride=1
w=math.floor(7*24*60*60/dt)
length = bikes.size - w - lag * w - q
print(bikes.size, w, lag, q, length)
XX=bikes[q:q+length:stride]
#week
for i in range(1,lag):
    X=bikes[i*w+q:i*w+q+length:stride]
    XX=np.column_stack((XX,X))
d=math.floor(24*60*60/dt)
#days
for i in range(0,lag):
    X=bikes[i*d+q:i*d+q+length:stride]
    XX=np.column_stack((XX,X))

for i in range(0,lag):
    X=bikes[i:i+length:stride]
    XX=np.column_stack((XX,X))

yy=bikes[lag*w+w+q:lag*w+w+q+length:stride] 
tt=time[lag*w+w+q:lag*w+w+q+length:stride]

yy.reset_index(drop=True, inplace=True)
tt.reset_index(drop=True, inplace=True)


train, test = train_test_split(np.arange(0,yy.size),test_size=0.2)

coeff = []
alphas = []
errors = []
scores = []
C = 10
qrange = [1, 2, 3, 4,5]


c_temp = []
mean_error = []
std_error = []
for q in qrange:
    print(q)
    temp = []
    XPoly = PolynomialFeatures(q).fit_transform(XX)
    a = 1/(2*Ci)
    print(Ci, a)
    model = Lasso(alpha= a).fit(XPoly[train], yy[train])
    coeff.append(model.coef_)
    alphas.append(a)
    y_pred = model.predict(XPoly)
    score = model.score(XPoly[test], yy[test])
    #r2_score = r2_score(yy,y_pred)
    # scores = cross_val_score(model, XX[test], yy[test], cv=5, scoring='f1')
    mse = mean_squared_error(yy,y_pred)
    errors.append(np.sqrt(mse))
    r2 = r2_score(yy,y_pred)
    scores.append(r2)
    print("SCORE: {0:.3f}, MSE:{1:.2f}, RMSE:{2:.2f}"
   .format(score,  mse,np.sqrt(mse)))
    #print(r2_score(yy,y_pred))
    print(f'r2_score - {r2}')


plt.close("all")
fig, (ax1, ax2) = plt.subplots(1, 2)

ax1.plot(qrange,scores)
ax1.set_xlabel('Alpha (Polynomial range)')
ax1.set_ylabel('Beta (Score)')
ax1.set_title('Model Score vs Polynomial Value')
ax1.axis('tight')

ax2 = plt.gca()
ax2.ticklabel_format(useOffset=False)
errors = [round(num, 5) for num in errors]
ax2.plot(qrange, errors)
ax2.set_xlabel("ploynomial value")
ax2.set_ylabel("error")
ax2.set_title("Error vs polynomial")
ax2.axis("tight")
fig.show()

plt.show()


17127 2016 3 2 9061
1
0.01 50.0
SCORE: 0.095, MSE:71.56, RMSE:8.46
r2_score - 0.09236433484207496
2
0.01 50.0
SCORE: 0.454, MSE:45.38, RMSE:6.74
r2_score - 0.4244749046627091
3
0.01 50.0


C:\Users\rkhna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 61502.76291430325, tolerance: 56.36990149006628
  model = cd_fast.enet_coordinate_descent(


SCORE: 0.627, MSE:32.00, RMSE:5.66
r2_score - 0.5940949568115446
4
0.01 50.0


C:\Users\rkhna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 99047.6351269287, tolerance: 56.36990149006628
  model = cd_fast.enet_coordinate_descent(


SCORE: 0.681, MSE:26.83, RMSE:5.18
r2_score - 0.659754057441892
5
0.01 50.0
SCORE: 0.702, MSE:24.82, RMSE:4.98
r2_score - 0.685153232219607


C:\Users\rkhna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 92961.82439871346, tolerance: 56.36990149006628
  model = cd_fast.enet_coordinate_descent(


In [33]:
# individual final run


import math
from sklearn.metrics import r2_score
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures

#%matplotlib inline
%matplotlib qt


t_full = pd.array(pd.DatetimeIndex(suburb_df.iloc[:,1]).astype(np.int64))/1000000000
dt = t_full[1]-t_full[0]

def get_feature_data(q, lag, stride, bikes, time, dt):
    w=math.floor(7*24*60*60/dt)
    length = bikes.size - w - lag * w - q
    print(bikes.size, w, lag, q, length)
    XX=bikes[q:q+length:stride]
    #week
    for i in range(1,lag):
        X=bikes[i*w+q:i*w+q+length:stride]
        XX=np.column_stack((XX,X))
    d=math.floor(24*60*60/dt)
    #days
    for i in range(0,lag):
        X=bikes[i*d+q:i*d+q+length:stride]
        XX=np.column_stack((XX,X))

    for i in range(0,lag):
        X=bikes[i:i+length:stride]
        XX=np.column_stack((XX,X))

    yy=bikes[lag*w+w+q:lag*w+w+q+length:stride] 
    tt=time[lag*w+w+q:lag*w+w+q+length:stride]

    yy.reset_index(drop=True, inplace=True)
    tt.reset_index(drop=True, inplace=True)
    return XX, yy, tt


def run_model(q, lag, stride, Ci, poly):
    plt.close("all")
    fig, ax = plt.subplots()
    XX, yy, tt = get_feature_data(q, lag, stride, bikes, time, dt)
    train, test = train_test_split(np.arange(0,yy.size),test_size=0.2)

    XPoly = PolynomialFeatures(poly).fit_transform(XX)
    a = 1/(2*Ci)
    print(f'Alpha & Ci  Value - {a} {Ci}')
    print(f"q value - {q}")
    model = Lasso(alpha= a).fit(XPoly[train], yy[train])
    y_pred = model.predict(XPoly)
    score = model.score(XPoly[test], yy[test])
    #r2_score = r2_score(yy,y_pred)
    mse = mean_squared_error(yy,y_pred)
    print("SCORE: {0:.3f}, MSE:{1:.2f}, RMSE:{2:.2f}" .format(score,  mse,np.sqrt(mse)))
    r2 = r2_score(yy,y_pred)
    print(f'r2_score : {r2}\n')
    ax.scatter(time, bikes, color="blue")
    ax.scatter(tt, y_pred, color="green") 
    ax.set_xlabel("time (days)")
    ax.set_ylabel("#bikes") 
    ax.set_title(f"Ridge model for q = {q}")
    ax.legend(["training data","predictions"],loc="upper right") 
    fig.show()

q = 12
lag = 3
stride = 1
Ci = 10
poly = 5
run_model(q, lag, stride, Ci, poly)

C:\Users\rkhna\AppData\Local\Temp/ipykernel_33108/1256422414.py:16: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  t_full = pd.array(pd.DatetimeIndex(suburb_df.iloc[:,1]).astype(np.int64))/1000000000


17127 2016 3 12 9051
Alpha & Ci  Value - 0.05 10
q value - 12
SCORE: 0.691, MSE:21.44, RMSE:4.63
r2_score : 0.7281398571130183



C:\Users\rkhna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 77217.18343762239, tolerance: 58.2294695994475
  model = cd_fast.enet_coordinate_descent(


In [36]:
#final run with all identified parameters

import math
from sklearn.metrics import r2_score
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures

#%matplotlib inline
%matplotlib qt


t_full = pd.array(pd.DatetimeIndex(suburb_df.iloc[:,1]).astype(np.int64))/1000000000
dt = t_full[1]-t_full[0]

def get_feature_data(q, lag, stride, bikes, time, dt):
    w=math.floor(7*24*60*60/dt)
    length = bikes.size - w - lag * w - q
    print(bikes.size, w, lag, q, length)
    XX=bikes[q:q+length:stride]
    #week
    for i in range(1,lag):
        X=bikes[i*w+q:i*w+q+length:stride]
        XX=np.column_stack((XX,X))
    d=math.floor(24*60*60/dt)
    #days
    for i in range(0,lag):
        X=bikes[i*d+q:i*d+q+length:stride]
        XX=np.column_stack((XX,X))

    for i in range(0,lag):
        X=bikes[i:i+length:stride]
        XX=np.column_stack((XX,X))

    yy=bikes[lag*w+w+q:lag*w+w+q+length:stride] 
    tt=time[lag*w+w+q:lag*w+w+q+length:stride]

    yy.reset_index(drop=True, inplace=True)
    tt.reset_index(drop=True, inplace=True)
    return XX, yy, tt

qrange = [ 2, 6, 12]
lag = 3
stride = 1
plt.close("all")
fig, ax = plt.subplots(3)
ax_index = 0
Ci = 10
poly = 5
q_score = {}
xlim = -10
ylim = 45 # 40 for Grangegorman
for q in qrange:
    XX, yy, tt = get_feature_data(q, lag, stride, bikes, time, dt)
    train, test = train_test_split(np.arange(0,yy.size),test_size=0.2)

    XPoly = PolynomialFeatures(poly).fit_transform(XX)
    a = 1/(2*Ci)
    print(f'Alpha & Ci  Value - {a} {Ci}')
    print(f"q value - {q}")
    model = Lasso(alpha= a).fit(XPoly[train], yy[train])
    y_pred = model.predict(XPoly)
    score = model.score(XPoly[test], yy[test])
    #r2_score = r2_score(yy,y_pred)
    mse = mean_squared_error(yy,y_pred)
    print("SCORE: {0:.3f}, MSE:{1:.2f}, RMSE:{2:.2f}" .format(score,  mse,np.sqrt(mse)))
    r2 = r2_score(yy,y_pred)
    print(f'r2_score - {r2}\n')
    q_score[q] = r2

#     plt.close("all")
#     fig, ax = plt.subplots()

    ax[ax_index].scatter(time, bikes, color="blue")
    ax[ax_index].scatter(tt, y_pred, color="green") 
    ax[ax_index].set_xlabel("time (days)")
    ax[ax_index].set_ylabel("#bikes") 
    ax[ax_index].set_title(f"Ridge model for q = {q}")
    ax[ax_index].legend(["training data","predictions"],loc="upper right")
    ax[ax_index].set_ylim([xlim, ylim])
    ax_index += 1
print(({"-"*30}))
for q, score in q_score.items():
    print(f"q value {q} - score {score}")
fig.show()

C:\Users\rkhna\AppData\Local\Temp/ipykernel_33108/986339219.py:15: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  t_full = pd.array(pd.DatetimeIndex(suburb_df.iloc[:,1]).astype(np.int64))/1000000000


17127 2016 3 2 9061
Alpha & Ci  Value - 0.05 10
q value - 2


C:\Users\rkhna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 84123.70677216415, tolerance: 58.01227411699779
  model = cd_fast.enet_coordinate_descent(


SCORE: 0.701, MSE:22.87, RMSE:4.78
r2_score - 0.7099794839604953

17127 2016 3 6 9057
Alpha & Ci  Value - 0.05 10
q value - 6


C:\Users\rkhna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 81510.20548248584, tolerance: 55.97829708764658
  model = cd_fast.enet_coordinate_descent(


SCORE: 0.743, MSE:22.26, RMSE:4.72
r2_score - 0.7177420707370492

17127 2016 3 12 9051
Alpha & Ci  Value - 0.05 10
q value - 12
SCORE: 0.722, MSE:21.44, RMSE:4.63
r2_score - 0.728183907900716

{'------------------------------'}
q value 2 - score 0.7099794839604953
q value 6 - score 0.7177420707370492
q value 12 - score 0.728183907900716


C:\Users\rkhna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 77416.1649523045, tolerance: 56.89585269337016
  model = cd_fast.enet_coordinate_descent(
